In [1]:
import py_at_broker as pab
import time
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline



In [2]:
def Rx(alpha):
    transform=np.eye(4)
    
    transform[1,1]=np.cos(alpha)
    transform[2,2]=np.cos(alpha)
    transform[2,1]=np.sin(alpha)
    transform[1,2]=-np.sin(alpha)
    return transform


def Rz(theta):
    transform=np.eye(4)
    
    transform[0,0]=np.cos(theta)
    transform[1,1]=np.cos(theta)
    transform[1,0]=np.sin(theta)
    transform[0,1]=-np.sin(theta)
    return transform


def Transl(x,y, z):
    transform=np.eye(4)
    
    transform[0,3]=x
    transform[1,3]=y
    transform[2,3]=z
    
    return transform

#def HDtransfromation(d,theta,a, alpha): # HD stands for hartenberg-denavit:
def HDtransfromation(params): # HD stands for hartenberg-denavit:
    d = params['d']
    theta = params['theta']
    a = params['a']
    alpha = params['alpha']
    #return np.dot(Transl(0,0,d), np.dot(Rx(alpha), np.dot(Transl(a, 0, 0), Rz(theta))))
    return np.dot(Rx(alpha), np.dot(Transl(a, 0, 0), np.dot(Rz(theta), Transl(0,0,d))))
    
def Transform2Base(df): 
    single_trans_list = [HDtransfromation(dict(df.loc[i])) for i in range(len(df))]

    total = []
    total.append(single_trans_list[0])
    for i in range(1, len(single_trans_list)): 
        total.append(np.dot(total[i-1], single_trans_list[i]))
    return total


### These are the HD parameters



In [3]:
A=np.array([ 0,0,0,0.0825,-0.0825,0,0.088,0])
D=np.array([ 0.333,0,0.316,0,0.384,0,0,0.107])
Alph=(np.pi/2)*(np.array([0,-1,1,1,-1,1,1,0]))
Thet=np.zeros(8)
#Thet[:7] = np.nan
d = {'a':A, 'd':D,'alpha':Alph, 'theta': Thet}
df = pd.DataFrame(d)
df

,a,d,alpha,theta
0,0.0000,0.333,0.000000,0.0
1,0.0000,0.000,-1.570796,0.0
2,0.0000,0.316,1.570796,0.0
3,0.0825,0.000,1.570796,0.0
4,-0.0825,0.384,-1.570796,0.0
5,0.0000,0.000,1.570796,0.0
6,0.0880,0.000,1.570796,0.0
7,0.0000,0.107,0.000000,0.0


### Positions of the joints where all thetas are equal to 0



In [4]:
for ourmatrix in Transform2Base(df):
    print(np.round(np.dot(ourmatrix , np.array((0,0,0,1)) ), 3))

[0.    0.    0.333 1.   ]
[0.    0.    0.333 1.   ]
[0.    0.    0.649 1.   ]
[0.082 0.    0.649 1.   ]
[0.    0.    1.033 1.   ]
[0.    0.    1.033 1.   ]
[0.088 0.    1.033 1.   ]
[ 0.088 -0.     0.926  1.   ]


In [5]:
for i in range(len(df)): 
    print(np.round(HDtransfromation(dict(df.loc[i])), 3))
    print('\r')

[[1.    0.    0.    0.   ]
 [0.    1.    0.    0.   ]
 [0.    0.    1.    0.333]
 [0.    0.    0.    1.   ]]

[[ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  0.  1.]]

[[ 1.     0.     0.     0.   ]
 [ 0.     0.    -1.    -0.316]
 [ 0.     1.     0.     0.   ]
 [ 0.     0.     0.     1.   ]]

[[ 1.     0.     0.     0.082]
 [ 0.     0.    -1.     0.   ]
 [ 0.     1.     0.     0.   ]
 [ 0.     0.     0.     1.   ]]

[[ 1.     0.     0.    -0.082]
 [ 0.     0.     1.     0.384]
 [ 0.    -1.     0.     0.   ]
 [ 0.     0.     0.     1.   ]]

[[ 1.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]]

[[ 1.     0.     0.     0.088]
 [ 0.     0.    -1.     0.   ]
 [ 0.     1.     0.     0.   ]
 [ 0.     0.     0.     1.   ]]

[[1.    0.    0.    0.   ]
 [0.    1.    0.    0.   ]
 [0.    0.    1.    0.107]
 [0.    0.    0.    1.   ]]

